In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/bond_etf_yield.csv', index_col=0)
df

,Name,Yield to Maturity,Date
Ticker,,,
AGG,iShares Core U.S. Aggregate Bond ETF,0.0506,08-14-2023
TLT,iShares 20+ Year Treasury Bond ETF,0.0440,08-14-2023
LQD,iShares iBoxx $ Investment Grade Corporate Bon...,0.0565,08-14-2023
MUB,iShares National Muni Bond ETF,0.0348,08-14-2023
IEF,iShares 7-10 Year Treasury Bond ETF,0.0422,08-14-2023
...,...,...,...
MBSD,FLEXSHARES DISCIPLINED DURATION MBS INDEX FUND,0.0545,08-15-2023
RAVI,FLEXSHARES ULTRA-SHORT INCOME FUND,0.0591,08-15-2023
SKOR,FLEXSHARES CREDIT-SCORED US CORPORATE BOND IND...,0.0581,08-15-2023


In [3]:
df.sort_values(by='Yield to Maturity', ascending=False).dropna()

,Name,Yield to Maturity,Date
Ticker,,,
PGHY,Invesco Global Short Term High Yield Bond ETF,0.1022,08-11-2023
JBBB,B-BBB CLO ETF,0.0973,08-15-2023
HYGV,FLEXSHARES HIGH YIELD VALUE-SCORED BOND INDEX ...,0.0971,08-15-2023
EMHY,iShares J.P. Morgan EM High Yield Bond ETF,0.0960,08-14-2023
BKLN,Invesco Senior Loan ETF,0.0947,08-11-2023
...,...,...,...
BBIP,JPMorgan BetaBuilders U.S. TIPS 0-5 Year ETF,0.0254,07-31-2023
TIP,iShares TIPS Bond ETF,0.0237,08-14-2023
PBTP,Invesco PureBeta℠ 0-5 Yr US TIPS ETF,0.0218,08-11-2023


In [4]:
df.loc['HYGV']

Name                 FLEXSHARES HIGH YIELD VALUE-SCORED BOND INDEX ...
Yield to Maturity                                               0.0971
Date                                                        08-15-2023
Name: HYGV, dtype: object

In [16]:
from yahooquery import Ticker
from arch import arch_model
from tqdm import tqdm
import pandas as pd
import numpy as np
from arch.__future__ import reindexing

def download_stock_data(tickers, period='3y'):
    ticker_str = ' '.join(tickers)
    ticker_obj = Ticker(ticker_str, asynchronous=True)
    stock_data = ticker_obj.history(period=period)
    
    return stock_data

def get_standard_deviation(stock_data):
    std_devs = {}
    for symbol in stock_data.index.get_level_values('symbol').unique():
        symbol_data = stock_data.loc[symbol]
        returns = symbol_data['close'].pct_change().dropna()
        annualized_std_dev = returns.std() * np.sqrt(252)
        std_devs[symbol] = annualized_std_dev
    
    return round(pd.DataFrame.from_dict(std_devs, orient='index', columns=['Standard Deviation']), 4)

# Function to tune GARCH parameters
def tune_garch_parameters(returns):
    best_aic = np.inf
    best_order = None
    p_values = range(1, 3)
    q_values = range(1, 3)
    for p in p_values:
        for q in q_values:
            try:
                model = arch_model(returns, vol='Garch', p=p, o=0, q=q, rescale=False)
                res = model.fit(disp='off')
                if res.aic < best_aic:
                    best_aic = res.aic
                    best_order = (p, q)
            except:
                continue
    return best_order

def get_expected_standard_deviation(stock_data):
    expected_std_devs = {}
    for symbol in tqdm(stock_data.index.get_level_values('symbol').unique()):
        symbol_data = stock_data.loc[symbol]
        returns = symbol_data['close'].pct_change().dropna() * 100

        # Call the tuning function to get the best p and q
        try:
            best_p, best_q = tune_garch_parameters(returns)
        except TypeError:
            continue

        # Fit a GARCH model with the best p and q
        model = arch_model(returns, vol='Garch', p=best_p, o=0, q=best_q, rescale=False)
        res = model.fit(disp='off')

        # Get the last forecast of the conditional volatility and annualize it
        forecast = res.forecast(start=0).variance.iloc[-1][0] ** 0.5 * np.sqrt(252)

        # Rescale the forecast back to the original scale
        expected_std_dev = forecast / 100

        # Store the result
        expected_std_devs[symbol] = round(expected_std_dev, 4)

    # Return the results as a DataFrame
    return pd.DataFrame.from_dict(expected_std_devs, orient='index', columns=['Expected Annualized Std Dev'])


In [18]:
# Assuming you have a DataFrame df that has the tickers as the index
tickers = list(df.index)
stock_data_dict = download_stock_data(tickers)
std_dev_df = get_standard_deviation(stock_data_dict)
expected_std_dev_df = get_expected_standard_deviation(stock_data_dict)
result_df = std_dev_df.join(expected_std_dev_df)
result_df.index.name = 'Ticker'
result_df

,Standard Deviation,Expected Annualized Std Dev
Ticker,,
AGG,0.0615,0.0689
TLT,0.1676,0.1647
LQD,0.0924,0.0925
MUB,0.0364,0.0404
IEF,0.0807,0.0872
...,...,...
MBSD,0.0454,0.0574
RAVI,0.0188,0.0149
SKOR,0.0472,0.0495


In [39]:
final_df = pd.concat([df, result_df], axis=1).dropna()
final_df['Yield to Volatility'] = round(final_df['Yield to Maturity'] / final_df['Expected Annualized Std Dev'], 2)
final_df['P/E Ratio'] = round(1 / final_df['Yield to Maturity'], 2)
final_df = final_df[[
    'Name', 'Yield to Maturity', 'Expected Annualized Std Dev', 'P/E Ratio', 'Yield to Volatility', 'Date'
]]

In [40]:
filtered_df = final_df.sort_values(by='Yield to Volatility', ascending=False).head(25).sort_values(by='Yield to Maturity', ascending=False)
filtered_df['Yield to Maturity'] = (filtered_df['Yield to Maturity'] * 100).apply(lambda x: f"{x:.2f}%")
filtered_df['Expected Annualized Std Dev'] = (filtered_df['Expected Annualized Std Dev'] * 100).apply(lambda x: f"{x:.2f}%")
filtered_df

,Name,Yield to Maturity,Expected Annualized Std Dev,P/E Ratio,Yield to Volatility,Date
Ticker,,,,,,
JAAA,AAA CLO ETF,7.36%,1.77%,13.59,4.16,08-15-2023
VRIG,Invesco Variable Rate Investment Grade ETF,7.30%,1.47%,13.70,4.97,08-11-2023
NEAR,BlackRock Short Maturity Bond ETF,6.28%,1.15%,15.92,5.46,08-14-2023
FLOT,iShares Floating Rate Bond ETF,6.26%,1.34%,15.97,4.67,08-14-2023
GSY,Invesco Ultra Short Duration ETF,6.06%,1.08%,16.50,5.61,08-11-2023
IBDP,iShares® iBonds® Dec 2024 Term Corporate ETF,5.95%,1.50%,16.81,3.97,08-14-2023
JPST,JPMorgan Ultra-Short Income ETF,5.95%,1.24%,16.81,4.80,08-15-2023
RAVI,FLEXSHARES ULTRA-SHORT INCOME FUND,5.91%,1.49%,16.92,3.97,08-15-2023
GSST,Goldman Sachs Access Ultra Short Bond ETF,5.84%,1.27%,17.12,4.60,08-15-2023
